In [1]:
print("""
@Description: How to Think About Group Operations
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-06 17:38:50
""")


@Description: How to Think About Group Operations
@Author(s): Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime: 2023-08-06 17:38:50



# How to Think About Group Operations

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.792178,0.409665
1,a,2,-1.289033,-1.433261
2,None,1,1.500917,-0.938335
3,b,2,0.830434,-0.108840
4,b,1,-0.943498,0.504764
5,a,<NA>,0.495648,-0.643027
6,None,1,-0.370578,-1.510295
